# Regressione Lineare con Spark MLlib

### In questo notebook vedremo come eseguire una semplice regressione, utilizzando un modello di Regressione Lineare, con il modulo MLlib di Spark. Il modello che andremo a creare avrà lo scopo di stimare il valore di un'abitazione utilizzando un set di proprietà:

#### CRIM Tasso di criminalità per capita
#### ZN Percentuale di terreni residenziali suddivisi in zone per lotti superiori a 25.000 sq.ft.
#### INDUS Percentuale di ettari di attività non al dettaglio per città.
#### CHAS Variabile dummy che indica la prossimità al fiume Charles.
#### NOX Concentrazione di ossido d'azoto (parti per 10 milioni).
#### RM Numero medio di stanze per abitazione
#### AGE Percentuale di abitazione occupate costruite dopo il 1940
#### DIS Media pesata delle distanze da 5 centri lavorativi di Boston.
#### RAD Indice di accessibilità ad autostrade
#### TAX Aliquota dell'imposta sulla proprietà a valore pieno in 10.000 USD.
#### PRATIO Rapporto studente-insegnante per città.
#### BLACK 1000(Bk - 0.63)^2 dove Bk è la percentuale di abitanti di colore per città
#### LSTAT Percentuale della popolazione povera
#### MEDV Mediana del valore di abitazioni occupate in 1.000 USD.

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("regression").getOrCreate()

In [4]:
housing_df = spark.read.csv("./data/houses_train.csv", inferSchema=True, header=True)
housing_df.show(5)

+---+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+
| ID|   crim|  zn|indus|chas|  nox|   rm| age|   dis|rad|tax|ptratio| black|lstat|medv|
+---+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+
|  1|0.00632|18.0| 2.31|   0|0.538|6.575|65.2|  4.09|  1|296|   15.3| 396.9| 4.98|24.0|
|  2|0.02731| 0.0| 7.07|   0|0.469|6.421|78.9|4.9671|  2|242|   17.8| 396.9| 9.14|21.6|
|  4|0.03237| 0.0| 2.18|   0|0.458|6.998|45.8|6.0622|  3|222|   18.7|394.63| 2.94|33.4|
|  5|0.06905| 0.0| 2.18|   0|0.458|7.147|54.2|6.0622|  3|222|   18.7| 396.9| 5.33|36.2|
|  7|0.08829|12.5| 7.87|   0|0.524|6.012|66.6|5.5605|  5|311|   15.2| 395.6|12.43|22.9|
+---+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+
only showing top 5 rows



# Preprocessing dei dati

In [16]:
features_cols = housing_df.columns[1:-1]
print(features_cols)

['crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis', 'rad', 'tax', 'ptratio', 'black', 'lstat']


In [20]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=features_cols, outputCol="features")
data_df = assembler.transform(housing_df)
data_df.show(5)

+---+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+--------------------+
| ID|   crim|  zn|indus|chas|  nox|   rm| age|   dis|rad|tax|ptratio| black|lstat|medv|            features|
+---+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+--------------------+
|  1|0.00632|18.0| 2.31|   0|0.538|6.575|65.2|  4.09|  1|296|   15.3| 396.9| 4.98|24.0|[0.00632,18.0,2.3...|
|  2|0.02731| 0.0| 7.07|   0|0.469|6.421|78.9|4.9671|  2|242|   17.8| 396.9| 9.14|21.6|[0.02731,0.0,7.07...|
|  4|0.03237| 0.0| 2.18|   0|0.458|6.998|45.8|6.0622|  3|222|   18.7|394.63| 2.94|33.4|[0.03237,0.0,2.18...|
|  5|0.06905| 0.0| 2.18|   0|0.458|7.147|54.2|6.0622|  3|222|   18.7| 396.9| 5.33|36.2|[0.06905,0.0,2.18...|
|  7|0.08829|12.5| 7.87|   0|0.524|6.012|66.6|5.5605|  5|311|   15.2| 395.6|12.43|22.9|[0.08829,12.5,7.8...|
+---+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+--------------------+
only showing top 5 

In [22]:
data_df.select("features").show(5, False)

+--------------------------------------------------------------------------+
|features                                                                  |
+--------------------------------------------------------------------------+
|[0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.09,1.0,296.0,15.3,396.9,4.98]   |
|[0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.9,9.14]  |
|[0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94] |
|[0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.9,5.33]  |
|[0.08829,12.5,7.87,0.0,0.524,6.012,66.6,5.5605,5.0,311.0,15.2,395.6,12.43]|
+--------------------------------------------------------------------------+
only showing top 5 rows



#  Normalizzazione

E' buona norma portare le features in un range di valori comuni, questo processo può velocizzare anche di molto la fase di addestramento. Facciamolo tramite la normalizzazione che si esegue sottraendo il valore minimo e poi dividendo per la differenza tra valore massimo e valore minimo. Possiamo eseguire la normalizzazione con MLlib usando la classe MinMaxScaler.

In [25]:
from pyspark.ml.feature import MinMaxScaler

scaler = MinMaxScaler(inputCol="features", outputCol="scaled_features")
scaler_model = scaler.fit(data_df)
data_df = scaler_model.transform(data_df)

data_df.show(5)

+---+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+--------------------+--------------------+
| ID|   crim|  zn|indus|chas|  nox|   rm| age|   dis|rad|tax|ptratio| black|lstat|medv|            features|     scaled_features|
+---+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+--------------------+--------------------+
|  1|0.00632|18.0| 2.31|   0|0.538|6.575|65.2|  4.09|  1|296|   15.3| 396.9| 4.98|24.0|[0.00632,18.0,2.3...|[0.0,0.18,0.05814...|
|  2|0.02731| 0.0| 7.07|   0|0.469|6.421|78.9|4.9671|  2|242|   17.8| 396.9| 9.14|21.6|[0.02731,0.0,7.07...|[2.85470335157677...|
|  4|0.03237| 0.0| 2.18|   0|0.458|6.998|45.8|6.0622|  3|222|   18.7|394.63| 2.94|33.4|[0.03237,0.0,2.18...|[3.54287862356241...|
|  5|0.06905| 0.0| 2.18|   0|0.458|7.147|54.2|6.0622|  3|222|   18.7| 396.9| 5.33|36.2|[0.06905,0.0,2.18...|[8.53146933036738...|
|  7|0.08829|12.5| 7.87|   0|0.524|6.012|66.6|5.5605|  5|311|   15.2| 395.6|12.43|22.9|[0.

In [26]:
data_df.select("scaled_features").show(5, False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|scaled_features                                                                                                                                                                                                                   |
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[0.0,0.18,0.05814814814814815,0.0,0.3148148148148149,0.5836560805577072,0.6297872340425532,0.30899621113279824,0.0,0.20650095602294455,0.31395348837209314,1.0,0.08967991169977926]                                               |
|[2.854703351576779E-4,0.0,0.23444444444444446,0.0,0.17283950617283944,0.55383423702

# Creazione modello di regressione lineare

Prossimo passo, dividere il DataFrame con le features preprocessate in due DataFrame, uno per l'addestramento e uno per il testing del modello, possiamo farlo utilizzando il metodo randomSplit all'interno della quale dobbiamo passare una lista con la percentuale di osservazioni da assegnare ad ognuno dei DataFrame.
Nel nostro caso assegnamo il 70% degli esempi al set di addestramento e il 30% al set di test.


In [27]:
train_df, test_df = data_df.randomSplit([0.7, 0.3], seed=0)

In [28]:
train_df.count()

243

In [30]:
test_df.count()

90

In [32]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(featuresCol="scaled_features", labelCol="medv")

In [34]:
model = lr.fit(train_df)

# Valutiamo il modello

In [41]:
evaluation = model.evaluate(test_df)
type(evaluation)

pyspark.ml.regression.LinearRegressionSummary

# MSE - Mean Squared Error (Errore quadratico assoluto)

L'errore quadratico medio consiste nella media della somma degli errori al quadrato.


In [43]:
evaluation.meanSquaredError

30.444470980244144

# RMSE - Root Mean Squared Error (Radice dell'errore quadratico medio)

Il RMSE è la radice dell'errore quadratico medio, questa metrica indica mediamente di quanto il nostro modello si è sbagliato.

In [44]:
evaluation.rootMeanSquaredError

5.517650857044521

Il modello  sbaglia mediamente di 55000 dollari

# MAE - Mean Absolute Error (Errore medio assoluto)

L'errore medio assoluto consiste nella media della somma del valore assoluto degli errori.

In [45]:
evaluation.meanAbsoluteError

3.7370129398441114

Il modello sbaglia mediamente di 37000 dollari

# R2 - Coefficient of determination (Coefficiente di Determinazione)

In pratica R2 (pronuciato R Squared) è una versione standardizzata del MSE che torna un punteggio compreso tra 0 e 1 per il train set, mentre per il test set può assumere anche valori negativi. Essendo una funzione ma di scoring, un suo valore maggiore indica una qualità migliore del modello, il suo valore può essere così interpretato:

    R2_score < 0.3 il modello è inutile.
    0.3 < R2_score < 0.5 il modello è scarso.
    0.5 < R2_score < 0.7 il modello è discreto.
    0.7 < R2_score < 0.9 il modello è buono.
    0.9 < R2_score < 1 il modello è ottimo.
    R2_score = 1 molto probabilmente c'è un errore nel modello.


In [47]:
evaluation.r2

0.6805998721105677

# Testiamo il Modello

Ora che abbiamo un modello addestrato e funzionante testiamolo su nuovi dati, mettiamo caso che un'agenzia immobiliare ti abbia mandato un file CSV con le proprietà di 10 abitazioni, per la quale tu devi stimare il prezzo usando il modello che hai addestrato. Iniziamo scaricando il file CSV.


In [50]:
houses_df = spark.read.csv("./data/houses_test.csv", inferSchema=True, header=True)
houses_df.show(5)

+---+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+
| ID|   crim|  zn|indus|chas|  nox|   rm|  age|   dis|rad|tax|ptratio| black|lstat|
+---+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+
|  3|0.02729| 0.0| 7.07|   0|0.469|7.185| 61.1|4.9671|  2|242|   17.8|392.83| 4.03|
|  6|0.02985| 0.0| 2.18|   0|0.458| 6.43| 58.7|6.0622|  3|222|   18.7|394.12| 5.21|
|  8|0.14455|12.5| 7.87|   0|0.524|6.172| 96.1|5.9505|  5|311|   15.2| 396.9|19.15|
|  9|0.21124|12.5| 7.87|   0|0.524|5.631|100.0|6.0821|  5|311|   15.2|386.63|29.93|
| 10|0.17004|12.5| 7.87|   0|0.524|6.004| 85.9|6.5921|  5|311|   15.2|386.71| 17.1|
+---+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+
only showing top 5 rows



In [55]:
input_df = assembler.transform(houses_df)
input_df.show(5)

+---+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+--------------------+
| ID|   crim|  zn|indus|chas|  nox|   rm|  age|   dis|rad|tax|ptratio| black|lstat|            features|
+---+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+--------------------+
|  3|0.02729| 0.0| 7.07|   0|0.469|7.185| 61.1|4.9671|  2|242|   17.8|392.83| 4.03|[0.02729,0.0,7.07...|
|  6|0.02985| 0.0| 2.18|   0|0.458| 6.43| 58.7|6.0622|  3|222|   18.7|394.12| 5.21|[0.02985,0.0,2.18...|
|  8|0.14455|12.5| 7.87|   0|0.524|6.172| 96.1|5.9505|  5|311|   15.2| 396.9|19.15|[0.14455,12.5,7.8...|
|  9|0.21124|12.5| 7.87|   0|0.524|5.631|100.0|6.0821|  5|311|   15.2|386.63|29.93|[0.21124,12.5,7.8...|
| 10|0.17004|12.5| 7.87|   0|0.524|6.004| 85.9|6.5921|  5|311|   15.2|386.71| 17.1|[0.17004,12.5,7.8...|
+---+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+--------------------+
only showing top 5 rows



In [56]:
input_df = scaler_model.transform(input_df)
input_df.show(5)

+---+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+--------------------+--------------------+
| ID|   crim|  zn|indus|chas|  nox|   rm|  age|   dis|rad|tax|ptratio| black|lstat|            features|     scaled_features|
+---+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+--------------------+--------------------+
|  3|0.02729| 0.0| 7.07|   0|0.469|7.185| 61.1|4.9671|  2|242|   17.8|392.83| 4.03|[0.02729,0.0,7.07...|[2.85198329121319...|
|  6|0.02985| 0.0| 2.18|   0|0.458| 6.43| 58.7|6.0622|  3|222|   18.7|394.12| 5.21|[0.02985,0.0,2.18...|[3.20015101775138...|
|  8|0.14455|12.5| 7.87|   0|0.524|6.172| 96.1|5.9505|  5|311|   15.2| 396.9|19.15|[0.14455,12.5,7.8...|[0.00187996972028...|
|  9|0.21124|12.5| 7.87|   0|0.524|5.631|100.0|6.0821|  5|311|   15.2|386.63|29.93|[0.21124,12.5,7.8...|[0.00278697384852...|
| 10|0.17004|12.5| 7.87|   0|0.524|6.004| 85.9|6.5921|  5|311|   15.2|386.71| 17.1|[0.17004,12.5,7.8...|[0.00222664141

In [57]:
pred_df = model.transform(input_df)

In [58]:
pred_df.show(5)

+---+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+--------------------+--------------------+------------------+
| ID|   crim|  zn|indus|chas|  nox|   rm|  age|   dis|rad|tax|ptratio| black|lstat|            features|     scaled_features|        prediction|
+---+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+--------------------+--------------------+------------------+
|  3|0.02729| 0.0| 7.07|   0|0.469|7.185| 61.1|4.9671|  2|242|   17.8|392.83| 4.03|[0.02729,0.0,7.07...|[2.85198329121319...|29.967527267571267|
|  6|0.02985| 0.0| 2.18|   0|0.458| 6.43| 58.7|6.0622|  3|222|   18.7|394.12| 5.21|[0.02985,0.0,2.18...|[3.20015101775138...|24.811553114211552|
|  8|0.14455|12.5| 7.87|   0|0.524|6.172| 96.1|5.9505|  5|311|   15.2| 396.9|19.15|[0.14455,12.5,7.8...|[0.00187996972028...|17.602371609554382|
|  9|0.21124|12.5| 7.87|   0|0.524|5.631|100.0|6.0821|  5|311|   15.2|386.63|29.93|[0.21124,12.5,7.8...|[0.00278697384852...| 9.19

In [60]:
from pyspark.sql.functions import round

pred_df = pred_df.drop("features") \
                 .drop("scaled_features") \
                 .withColumn("estimated_price", round(pred_df["prediction"]*10000, 2)) \
                 .drop("prediction")